In [1]:
import gzip
import json
from pprint import pprint
from celan_app.models import Verse, Word, WordTranslation

In [2]:
verses = Verse.objects.all()

In [9]:
verse_pk = 110
print(verses[verse_pk].text)
verses[verse_pk].lemmas

ES STAND
der Feigensplitter auf deiner Lippe,

es stand
Jerusalem um uns,

es stand
der Hellkiefernduft
überm Dänenschiff, dem wir dankten,

ich stand
in dir.


{'du': 1,
 'es': 3,
 'in': 1,
 'um': 1,
 'auf': 1,
 'der': 3,
 'ich': 1,
 'wir': 2,
 'dein': 1,
 'Lippe': 1,
 'danken': 1,
 'stehen': 4,
 'überm': 1,
 'Jerusalem': 1,
 'Dänenschiff': 1,
 'Feigensplitter': 1,
 'Hellkiefernduft': 1}

In [20]:
def unify_lemmas():
    progress_check = range(0, 10_000_000, 100)
    verses = Verse.objects.all()
    unified = {}
    count = 0
    for i, verse in enumerate(verses):
        for word, number in verse.lemmas.items():
            unified.setdefault(word, [number]).append(number)
            count += 1
        if i in progress_check:
            print(i)
    print(f"number of unified lemmas = {len(unified)}")
    print(f"number of lemmas processed = {count}")
    return unified

In [21]:
unified = unify_lemmas()

0
100
200
300
400
500
600
700
number of unified lemmas = 8389
number of lemmas processed = 29530
